In [ ]:
import requests
from getpass import getpass
from bs4 import BeautifulSoup


headers = {
 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'
}

login_data = {
'commit': 'Sign in',
'utf8': '%E2%9C%93',
'login': input('Username: '),
'password': getpass()
}
url = 'https://github.com/session'
session = requests.Session()
response = session.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html5lib')
login_data['authenticity_token'] = soup.find(
'input', attrs={'name': 'authenticity_token'})['value']
response = session.post(url, data=login_data, headers=headers)
print(response.status_code)
response = session.get('https://github.com/settings/organizations', headers=headers)
# print(response.text)

In [2]:
response.headers

# from IPython.core.display import display, HTML
# display(HTML(response.text))

{'Date': 'Tue, 08 Sep 2020 12:39:17 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Server': 'GitHub.com', 'Status': '200 OK', 'Vary': 'X-PJAX, Accept-Encoding, Accept, X-Requested-With, Accept-Encoding', 'ETag': 'W/"cbad0a105c4944e190290f12c83c5502"', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Set-Cookie': 'user_session=UPayuSBdxptvXri5sWhZNQifTlldzWi6n8Cyp12k3nZpzWTC; path=/; expires=Tue, 22 Sep 2020 12:39:16 GMT; secure; HttpOnly; SameSite=Lax, __Host-user_session_same_site=UPayuSBdxptvXri5sWhZNQifTlldzWi6n8Cyp12k3nZpzWTC; path=/; expires=Tue, 22 Sep 2020 12:39:16 GMT; SameSite=Strict; secure; HttpOnly, has_recent_activity=1; path=/; expires=Tue, 08 Sep 2020 13:39:16 GMT; secure; HttpOnly; SameSite=Lax, _gh_sess=ubezCE20ojeTt%2BuCyJnBWdE1CvjCaO8w6ompvXeY54GI6YiY%2FM2zkHgYIcsWgat9tNlhN8to5fDp7CDW5KTVHOsBx9JkwzC%2F0HwQm6vxLWqOq0I9ZI3dqEZNlw1QOF3TYCLN00tFWJlwq1OGlsLKg2OIME%2F7DGX4ATC54QQuhc06ZjeZ0P5TrJTbhlITmvjL3lIyn27u47B9L1r2GgTspwJozr

In [3]:
soup = BeautifulSoup(response.text)

orgsSection = soup.findAll("div", {"class": "col-9 float-left"})[0].findAll("div", {"class": "Box"})[0].findAll('img')
myOrgs = "\n".join(set(tag['alt'] for tag in orgsSection))
print(myOrgs)


@diffusioncon
@NeverMindTheBlockchain
@blockpass-identity-lab
@OpenMined
